In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [2]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# 数据集 Dataset

## 1、添加一个Markdown单元格，在其中解释下方单元格的两行代码。
设置 os.environ['HF_ENDPOINT'] = \'https://hf-mirror.com' ，这样做具体改变了什么？
为什么要设置HF_ENDPOINT=\'https://hf-mirror.com'而非直接使用官方源？
dataset = datasets.load_dataset("bentrevett/multi30k") 这行代码具体完成了什么操作？

具体改变：
1. 默认资源下载地址从官方源 huggingface.co 切换为国内镜像站 hf-mirror.com
2. 此举主要解决国内用户因网络限制无法稳定访问官方源的问题，通过镜像站的国内服务器加速下载并避免超时；而 datasets.load_dataset("bentrevett/multi30k") 则是从 Hugging Face Hub 下载并加载名为 multi30k 的多语言机器翻译数据集

为何使用镜像而非官方源：
1. 国内用户可能无法稳定连接 `huggingface.co`，导致下载失败或速度极慢。
2. 镜像站与官方源文件结构完全一致，无需修改代码逻辑。

数据标识解析：
- bentrevett/multi30k 表示用户 bentrevett 上传的名为 multi30k 的数据集。

下载与缓存：
- 检查本地缓存（默认路径 ~/.cache/huggingface/datasets）是否已有该数据集。
- 若未缓存，则从 Hugging Face Hub（或通过 HF_ENDPOINT 设置的镜像源）下载数据文件。

加载数据：
- 将数据集解析为 Dataset 或 DatasetDict 对象，可直接用于训练或分析。

In [3]:
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
dataset = datasets.load_dataset("bentrevett/multi30k")

## 2、运行下方的单元格。
你会看到数据集对象（一个DatasetDict）包含训练、验证和测试集，每个集合中的样本数量，以及每个集合中的特征（“en”和“de”）。


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['en', 'de'],
        num_rows: 29000
    })
    validation: Dataset({
        features: ['en', 'de'],
        num_rows: 1014
    })
    test: Dataset({
        features: ['en', 'de'],
        num_rows: 1000
    })
})

In [6]:
train_data, valid_data, test_data = (
    dataset["train"],
    dataset["validation"],
    dataset["test"],
)

## 3、运行下方的单元格。
我们可以索引每个数据集来查看单个示例。每个例子都有两个特征：“en”和“de”，是对应的英语和德语。

In [7]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}

接下来我们进行分词。英语/德语的分词较中文要直接，比如句子"good morning!会被分词为["good", "morning", "!"]序列。

下方的代码要成功安装en_core_web_sm和de_core_news_sm后才不会报错。

# 分词器 Tokenizers

In [8]:
en_nlp = spacy.load("en_core_web_sm")
de_nlp = spacy.load("de_core_news_sm")

## 4、运行下方的单元格。
我们可以使用.tokenizer方法调用每个spaCy模型的分词器，该方法接受字符串并返回Token对象序列。我们可以使用text属性从Token对象中获取字符串。

In [9]:
string = "What a lovely day it is today!"

[token.text for token in en_nlp.tokenizer(string)]

['What', 'a', 'lovely', 'day', 'it', 'is', 'today', '!']

## 5、添加一个Markdown单元格，在其中解释下方单元格的函数的作用。

这个函数的作用是对输入的英语（en）和德语（de）文本进行分词和预处理。使用 en_nlp（英语分词器）对 example["en"] 进行分词，生成英语 token 列表 en_tokens。
使用 de_nlp（德语分词器）对 example["de"] 进行分词，生成德语 token 列表 de_tokens。通过 max_length 限制每个列表的最大长度（超出部分截断）。
在英语和德语的 token 列表首尾分别添加起始标记（sos_token）和结束标记（eos_token），用于标识句子边界。
输出返回包含处理后的 en_tokens 和 de_tokens 的字典，通常用于后续任务（如机器翻译模型的输入和目标序列）。

In [10]:
def tokenize_example(example, en_nlp, de_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["en"])][:max_length]
    de_tokens = [token.text for token in de_nlp.tokenizer(example["de"])][:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        de_tokens = [token.lower() for token in de_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    de_tokens = [sos_token] + de_tokens + [eos_token]
    return {"en_tokens": en_tokens, "de_tokens": de_tokens}

## 6、添加一个Markdown单元格，在其中解释下方单元格出现的\<sos>和\<eos>的含义，以及map函数的作用。

<sos>表示序列的起始标记，用于告诉模型“从这里开始生成或处理”。在机器翻译等序列生成任务中，解码器通常需要以 <sos> 作为初始输入，逐步生成后续内容。
<eos>表示序列的结束标记，用于告诉模型“处理到此为止”。模型在生成过程中若遇到 <eos> 会停止输出，避免生成无限长的序列。
map 函数的作用是对数据集（如 train_data、valid_data、test_data）中的每个样本（example）应用指定函数（此处为 tokenize_example），实现批量预处理。

In [11]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp": en_nlp,
    "de_nlp": de_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

## 7、运行下方的单元格
重新打印train_data\[0]，验证小写字符串列表以及序列标记的开始/结束符已被成功添加。

In [12]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

# 词汇表 Vocabularies

下一个步骤是为源语言和目标语言构建词汇表，将词语映射为数字索引。比如"hello" = 1, "world" = 2, "bye" = 3, "hates" = 4。当向我们的模型提供文本数据时，我们使用词汇表作为look-up-table将字符串转换为标记，然后将标记转换为数字。“hello world”变成了“\[“hello”，“world”]”，然后变成了“\[1,2]”。

In [13]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

de_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["de_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

## 8、运行下方两个单元格
验证词汇表，分别打印英语词汇表和德语词汇表的前十个Token。

In [14]:
en_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', 'a', '.', 'in', 'the', 'on', 'man']

In [15]:
de_vocab.get_itos()[:10]

['<unk>', '<pad>', '<sos>', '<eos>', '.', 'ein', 'einem', 'in', 'eine', ',']

## 9、运行下方的单元格
使用get_stoi（stoi = "string to int "）方法获取指定的Token的索引。

In [16]:
en_vocab["the"]

7

In [17]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]

In [18]:
en_vocab.set_default_index(unk_index)
de_vocab.set_default_index(unk_index)

词汇表的另一个有用特性是lookup_indices方法。它接受一个Token列表并返回一个索引列表。

## 10、运行下方的单元格
观察从Token列表到索引列表的转换。

In [19]:
tokens = ["i", "love", "watching", "crime", "shows"]
en_vocab.lookup_indices(tokens)

[956, 2169, 173, 0, 821]

对应的，lookup_tokens方法使用词汇表将索引列表转换回Token列表。

## 11、运行下方的单元格
观察从索引列表到Token列表的转换。

In [20]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', '<unk>', 'shows']

## 12、添加一个Markdown单元格，在其中解释为什么原本的"crime"被转换成了\<unk>。

en_vocab 的词汇表未包含 "crime"，导致 lookup_indices() 方法无法找到其对应的索引，从而返回 <unk> 的索引。

## 13、添加一个Markdown单元格，在其中解释下方两个单元格中代码的作用。

numericalize_example 函数的作用是将分词后的文本（如 ["<sos>", "hello", "world", "<eos>"]）转换为模型可处理的数值化索引序列（如 [1, 5, 3, 2]）。
map 函数调用的作用是对数据集（训练集、验证集、测试集）中的每个样本应用 numericalize_example 函数，批量完成数值化转换。

In [21]:
def numericalize_example(example, en_vocab, de_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    de_ids = de_vocab.lookup_indices(example["de_tokens"])
    return {"en_ids": en_ids, "de_ids": de_ids}

In [22]:
fn_kwargs = {"en_vocab": en_vocab, "de_vocab": de_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

## 14、运行下方的单元格
重新打印train_data\[0]，验证"en_ids" and "de_ids"被成功添加。

In [23]:
train_data[0]

{'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>'],
 'en_ids': [2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3],
 'de_ids': [2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]}

Dataset类为我们处理的另一件事是将features转换为正确的类型。每个例子中的索引目前都是基本的Python整数。然而，为了在PyTorch中使用它们，它们需要转换为PyTorch张量。with_format方法将columns参数转换为给定的类型。这里，我们指定类型为“torch”，columns为“en_ids”和“de_ids”（我们想要转换为PyTorch张量的features）。默认情况下，with_format将删除任何不在传递给列的features列表中的features。我们希望保留这些features，这可以通过output_all_columns=True来实现。

In [24]:
data_type = "torch"
format_columns = ["en_ids", "de_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

## 15、运行下方的单元格
重新打印train_data[0]，验证“en_ids”和“de_ids”特征被转换为了张量。

In [25]:
train_data[0]

{'en_ids': tensor([   2,   16,   24,   15,   25,  778,   17,   57,   80,  202, 1312,    5,
            3]),
 'de_ids': tensor([   2,   18,   26,  253,   30,   84,   20,   88,    7,   15,  110, 7647,
         3171,    4,    3]),
 'en': 'Two young, White males are outside near many bushes.',
 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.',
 'en_tokens': ['<sos>',
  'two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.',
  '<eos>'],
 'de_tokens': ['<sos>',
  'zwei',
  'junge',
  'weiße',
  'männer',
  'sind',
  'im',
  'freien',
  'in',
  'der',
  'nähe',
  'vieler',
  'büsche',
  '.',
  '<eos>']}

# Data Loaders

数据准备的最后一步是创建Data Loaders。可以对它们进行迭代以返回一批数据，每一批数据都是一个字典，其中包含数字化的英语和德语句子作为PyTorch张量。

## 16、添加一个Markdown单元格，在其中解释下方两个单元格中的函数的作用。

get_collate_fn 函数的作用是生成一个自定义的批处理函数（collate_fn），用于将多个样本的变长序列填充对齐，使它们可以被批量处理（如输入模型训练）。
get_data_loader 函数的作用是创建一个 PyTorch 数据加载器（DataLoader），用于按批次加载数据集，并应用自定义的 collate_fn 对齐数据。

In [26]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_de_ids = [example["de_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_de_ids = nn.utils.rnn.pad_sequence(batch_de_ids, padding_value=pad_index)
        batch = {
            "en_ids": batch_en_ids,
            "de_ids": batch_de_ids,
        }
        return batch

    return collate_fn

In [27]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [28]:
batch_size = 128

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

# 构建模型

我们将分三部分构建模型。编码器，解码器和封装编码器和解码器的seq2seq模型。

# 编码器 Encoder

首先是编码器，它是一个2层的LSTM。

## 17、添加一个Markdown单元格，解释下方单元格中Encoder类的代码。
包括输入参数，核心组件（词嵌入层、LSTM层、Dropout层），forwad函数的处理流程，和输出。

In [30]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src length, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src length, batch size, embedding dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [src length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # outputs are always from the top hidden layer
        return hidden, cell

# 解码器 Decoder

接下来是解码器，它需要与编码器对齐，同样是一个2层的LSTM。

## 18、添加一个Markdown单元格，描述Decoder的工作流程。

1.初始化参数与层定义
2. 前向传播流程 (forward)分为输入预处理（input = input.unsqueeze(0)  # [batch_size] → [1, batch_size]），词嵌入与正则化（embedded = self.dropout(self.embedding(input))  # [1, 64] → [1, 64, 256]），LSTM时序处理（output, (hidden, cell) = self.rnn(embedded, (hidden, cell))），预测输出生成（prediction = self.fc_out(output.squeeze(0))  # [1,64,512] → [64, 512] → [64, 10000]）
3.关键设计解析
4.状态传递机制
5.与Seq2Seq的协作

In [50]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim    # 目标词汇表大小（如英文词表）
        self.hidden_dim = hidden_dim    # 隐藏层维度（需与Encoder一致）
        self.n_layers = n_layers        # LSTM层数（需与Encoder一致）
        self.embedding = nn.Embedding(output_dim, embedding_dim)   # 词嵌入层：将离散词索引转为连续向量
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout)    # LSTM层：处理时序依赖关系
        self.fc_out = nn.Linear(hidden_dim, output_dim)     # 全连接层：将LSTM输出映射到目标词汇表维度
        self.dropout = nn.Dropout(dropout)       # Dropout层：防止过拟合

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hidden dim]
        # context = [n layers, batch size, hidden dim]
        input = input.unsqueeze(0)
        # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, embedding dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq length, batch size, hidden dim * n directions]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # seq length and n directions will always be 1 in this decoder, therefore:
        # output = [1, batch size, hidden dim]
        # hidden = [n layers, batch size, hidden dim]
        # cell = [n layers, batch size, hidden dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

# Seq2Seq

## 19、添加一个Markdown单元格，解释下方单元格中Seq2Seq类的代码。
包括forward函数的流程，以及teacher forcing机制。

类结构：
self.encoder = encoder    # 编码器组件
self.decoder = decoder    # 解码器组件
self.device = device      # 计算设备（CPU/GPU）
#确保编码器和解码器的隐藏层维度一致
assert encoder.hidden_dim == decoder.hidden_dim
#确保编码器和解码器的层数一致
assert encoder.n_layers == decoder.n_layers

Forward函数流程：输入维度处理，编码阶段，解码初始化，循环解码。

Teacher Forcing机制是动态输入选择，以teacher_forcing_ratio概率使用真实标签作为下一个输入，以1 - teacher_forcing_ratio概率使用模型预测结果。


In [42]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder    # 编码器组件
        self.decoder = decoder    # 解码器组件
        self.device = device      # 计算设备（CPU/GPU）
         # 确保编码器和解码器的隐藏层维度一致
        assert (
            encoder.hidden_dim == decoder.hidden_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"
         # 确保编码器和解码器的层数一致
        assert (
            encoder.n_layers == decoder.n_layers
        ), "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio):
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]   # 获取批次大小
        trg_length = trg.shape[0]   # 获取目标序列长度
        trg_vocab_size = self.decoder.output_dim  # 获取目标词汇表大小
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)   # 编码器处理源序列
        # hidden/cell形状：[num_layers, batch_size, hidden_dim]
        # hidden = [n layers * n directions, batch size, hidden dim]
        # cell = [n layers * n directions, batch size, hidden dim]
        # first input to the decoder is the <sos> tokens
        input = trg[0, :]   # 初始输入为<sos>标记
        # input = [batch size]
        for t in range(1, trg_length):
             # 解码步骤
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)  
            # output = [batch size, output dim]
            # hidden = [n layers, batch size, hidden dim]
            # cell = [n layers, batch size, hidden dim]
            # place predictions in a tensor holding predictions for each token
            outputs[t] = output  # 存储当前时间步的输出
            # decide if we are going to use teacher forcing or not
            # Teacher Forcing决策
            teacher_force = random.random() < teacher_forcing_ratio
            # get the highest predicted token from our predictions
            top1 = output.argmax(1)  # 获取预测词（贪婪搜索）
            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1  # 选择下一个输入
            # input = [batch size]
        return outputs

# 模型训练

模型初始化

## 20、添加注释
分别将“# 编码器初始化”，“# 解码器初始化”，“# Seq2Seq模型整合”这三行注释加到下方单元格中正确的位置

In [43]:
input_dim = len(de_vocab)
output_dim = len(en_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 编码器初始化
encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    hidden_dim,
    n_layers,
    encoder_dropout,
)
# 解码器初始化
decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    hidden_dim,
    n_layers,
    decoder_dropout,
)
# Seq2Seq模型整合
model = Seq2Seq(encoder, decoder, device).to(device)

权重初始化

In [34]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


优化器 optimizer

In [36]:
optimizer = optim.Adam(model.parameters())

损失函数 Loss Function

In [37]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

Training Loop:

## 21、给下方单元格中的代码逐行加注释

In [44]:
# 定义训练函数
def train_fn(
    model , data_loader,  optimizer, criterion, clip, teacher_forcing_ratio, device
): # 要训练的模型  # 数据加载器，用于批量加载数据  # 优化器，用于更新模型参数  # 损失函数，用于计算模型输出与目标之间的差异  # 梯度裁剪的阈值，防止梯度爆炸
     # 教师强制比例，控制使用真实标签作为输入的概率  # 计算设备（如'cuda'或'cpu'）
    model.train()   # 将模型设置为训练模式（这会启用dropout和batch normalization等训练特定行为）
    epoch_loss = 0      # 初始化epoch损失为0
    for i, batch in enumerate(data_loader):  # 遍历数据加载器中的每个批次
        src = batch["de_ids"].to(device)    # 获取源语言（德语）数据并移动到指定设备
        trg = batch["en_ids"].to(device)    # 获取目标语言（英语）数据并移动到指定设备
        # src = [src length, batch size]    # 注释说明张量形状：
        # trg = [trg length, batch size]
        optimizer.zero_grad()   # 清除优化器中之前的梯度
        output = model(src, trg, teacher_forcing_ratio)  # teacher_forcing_ratio控制是否使用真实标签作为解码器输入
        # output = [trg length, batch size, trg vocab size]   # 目标语言序列长度 × 批次大小 × 目标词汇表大小
        output_dim = output.shape[-1]  # 获取输出词汇表维度大小
        output = output[1:].view(-1, output_dim)  # 将输出展平为二维张量 [(trg length - 1) * batch size, trg vocab size]
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)    # 同样忽略第一个时间步，展平为一维张量
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)        # 计算损失：模型输出与真实标签之间的差异
        loss.backward()    # 反向传播：计算梯度
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)   # 梯度裁剪：防止梯度爆炸
        optimizer.step()    # 更新模型参数
        epoch_loss += loss.item()              # 累加当前批次的损失值
    return epoch_loss / len(data_loader)      # 返回整个epoch的平均损失

Evaluation Loop:

In [39]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_ids"].to(device)
            trg = batch["en_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

# 模型训练

In [40]:
n_epochs = 1 # 因模型训练对计算资源要求较高，此处只设立了一轮训练。
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "tut1-model.pt")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [15:59<00:00, 959.16s/it]

	Train Loss:   5.028 | Train PPL: 152.638
	Valid Loss:   4.885 | Valid PPL: 132.285


# 模型验证

In [45]:
model.load_state_dict(torch.load("tut1-model.pt"))

<All keys matched successfully>

In [46]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [47]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Ein Mann mit einem orangefarbenen Hut, der etwas anstarrt.',
 'A man in an orange hat starring at something.')

In [48]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

# 22、运行下方单元格，得到测试集第0个索引的翻译
因为epoch只进行了一轮，不会有好的效果的翻译。
感兴趣的同学可自行增加训练轮数，观察loss和翻译质量的变化。

In [49]:
translation

['<sos>',
 'a',
 'man',
 'in',
 'a',
 'a',
 'shirt',
 'is',
 'a',
 'a',
 'a',
 '.',
 '<eos>']